# Data Ingestion
This notebook is being used to research the data ingestion part of the project.

In [32]:
# Importing the OS Module
import os

In [33]:
## Changing the working directory
# %pwd
# os.chdir("../")
# %pwd

In [34]:
from dataclasses import dataclass
from pathlib import Path

# The return type of the functions are being defined here
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir = Path
    source_URL = str
    local_data_file = Path
    unzip_dir = Path

In [35]:
from PoultryGuard.constants import *
from PoultryGuard.utils.common import read_yaml, create_directories

In [36]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        # Read the YAML configuration files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Create directories as specified in the configuration
        create_directories([self.config.data_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [37]:
# Updating the components
import os
import urllib.request as request
import zipfile
from PoultryGuard import logger
from PoultryGuard.utils.common import get_size

In [38]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following info: \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [39]:
# Pipeline

try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-27 14:14:54,617: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-11-27 14:14:54,620: INFO: common: yaml file: params.yaml loaded successfully:]
[2024-11-27 14:14:54,621: INFO: common: Created directory at: data:]
[2024-11-27 14:14:54,622: INFO: common: Created directory at: data/data_ingestion:]


TypeError: DataIngestionConfig.__init__() got an unexpected keyword argument 'root_dir'